In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.preprocessing import MinMaxScaler, StandardScaler,scale, MaxAbsScaler # MaxAbs is process sparse data
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
from sklearn.model_selection import train_test_split

import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import nn, optim
import torch.nn.init as init
import torch.utils.data as Data
import math


import matplotlib.pyplot as plt
import torch.multiprocessing as mp
# add tuning 
from skorch import NeuralNetRegressor
from skorch.callbacks import EpochScoring


In [2]:
mp.set_start_method('spawn')

### default hyperparameters

In [3]:
batch_size = 128
use_gpu = True
y_scale = True
lr = 0.001
weight_decay = 0.0005

# Batch size and learning rate is hyperparameters in deep learning
# suggest batch_size is reduced, lr is also reduced which will reduce concussion

## using CUDA

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## reading file

In [5]:
X = pd.read_csv('./dataset-0510/train.csv')
X_test = pd.read_csv('./dataset-0510/test.csv')

columns = X.columns

In [6]:
y = X['total_price']
X = X.drop(columns=['building_id', 'total_price'], axis=1)
X_test = X_test.drop(columns=['building_id'], axis=1)

## define nn

In [7]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(233, 256)
        self.bn1 = nn.BatchNorm1d(num_features=256)
        
        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(num_features=512)
        
        self.fc3 = nn.Linear(512, 512)
        self.bn3 = nn.BatchNorm1d(num_features=512)
        
        self.fc4 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(num_features=256)
        
        self.fc5 = nn.Linear(256, 128)
        self.bn5 = nn.BatchNorm1d(num_features=128)
        
        self.fc6 = nn.Linear(128, 64)
        self.bn6 = nn.BatchNorm1d(num_features=64)
        
        self.fc7 = nn.Linear(64, 32)
        self.bn7 = nn.BatchNorm1d(num_features=32)
        
        self.fc8 = nn.Linear(32, 1)
        
        
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        #x = x.unsqueeze(0)
        
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.relu(self.bn4(self.fc4(x)))
        x = F.relu(self.bn5(self.fc5(x)))
        x = F.relu(self.bn6(self.fc6(x)))
        x = F.relu(self.bn7(self.fc7(x)))
        x = self.fc8(x)

        return x

## pipline

In [8]:
# step1. Imputation transformer for completing missing values.
step1 = ('Imputer', Imputer())
# step2. MinMaxScaler
step2 = ('MinMaxScaler', MinMaxScaler())
# step3. feature selection
#step3 = ('FeatureSelection', SelectFromModel(RandomForestRegressor()))
step3 = ('FeatureSelection', VarianceThreshold())

pipeline = Pipeline(steps=[step1, step2])

/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## preprocess

In [9]:

X = pipeline.fit_transform(X)

X_test = pipeline.transform(X_test)

y_scaler = MaxAbsScaler()
y = y_scaler.fit_transform(y.values.reshape(-1, 1))
# MaxAbs 0.00012
# MinMax 0.0008

In [10]:
print(X.shape)

print(X_test.shape)

print(y.shape)
y

(60000, 233)
(10000, 233)
(60000, 1)


array([[0.00012727],
       [0.00065277],
       [0.00188097],
       ...,
       [0.00232208],
       [0.00355425],
       [0.00167253]])

In [11]:
X = torch.from_numpy(X).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y = torch.from_numpy(y).float().to(device)

## define loss

In [11]:
class SqrtMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x, y):
        return torch.sqrt(torch.mean(torch.pow((x - y), 2)))

In [12]:
auc = EpochScoring(scoring='neg_mean_squared_error', lower_is_better=False)

net = NeuralNetRegressor(
    DNN,
    max_epochs=10,
    lr=0.001,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    callbacks=[auc],
    device='cuda',
    criterion= SqrtMSELoss,
)

#net.fit(X, y)
#y_proba = net.predict_proba(X)

In [ ]:
X = X.astype(np.float32)
y = y.astype(np.float32)

params = {
    'lr': [0.001, 0.0015, 0.003, 0.005],
    #'max_epochs': [2000, 5000],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='neg_mean_squared_error')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

  epoch    neg_mean_squared_error    train_loss    valid_loss     dur
-------  ------------------------  ------------  ------------  ------
      1                   -0.0075        0.1224        0.0864  1.1858
      2                   -0.0036        0.0734        0.0598  1.0619
      3                   -0.0022        0.0535        0.0467  1.0448
      4                   -0.0014        0.0421        0.0370  1.0511
      5                   -0.0010        0.0349        0.0311  1.0447
      6                   -0.0007        0.0295        0.0267  1.0447
      7                   -0.0005        0.0255        0.0229  1.0507
      8                   -0.0004        0.0225        0.0203  1.0507
      9                   -0.0003        0.0201        0.0182  1.0457
     10                   -0.0003        0.0181        0.0165  1.0379
  epoch    neg_mean_squared_error    train_loss    valid_loss     dur
-------  ------------------------  ------------  ------------  ------
      1             

      4                   -0.0003        0.0161        0.0152  1.0535
      5                   -0.0003        0.0136        0.0129  1.0524
      6                   -0.0002        0.0119        0.0117  1.0513
      7                   -0.0002        0.0108        0.0108  1.0604
      8                   -0.0002        0.0099        0.0100  1.0528


In [ ]:
y_proba = net.predict_proba(X_test)
pred = y_scaler.inverse_transform(y_proba)       
print(pred.shape)

In [30]:
submission = pd.read_csv('./dataset-0510/submit_test.csv')
submission['total_price'] = pred
submission.to_csv('submission/TuningDNN_result.csv', index=False)
submission

,building_id,total_price
0,X5gsdTWGS3W7JJQB,2.085470e+07
1,BTshNOJyKHnT2YIT,8.500902e+07
2,dhdymr0lV8N5kZOT,7.683752e+07
3,VEwyGGMcD56w5BOc,9.981914e+07
4,wmUeMoJZfsqaSX9b,5.877293e+06
5,EtBjGAHmHCe9t7TZ,1.415726e+07
6,hPNH34vmaZtvBtqc,1.381302e+07
7,wXjeI38bYDMJJwZC,-6.184460e+07
8,fxZSGX6aPAFKU8W4,-7.499238e+07
9,ewr0Fx6ign87OwaV,-6.231330e+07
